# Discussion 5

In this discussion we review the Derivative and integrals by focusing on their computational aspects. 

You can use the Shared Computing Cluster (SCC) or Google Colab to run this notebook.

The general instructions for running on the SCC are available under General Resources on [Piazza](https://piazza.com/bu/fall2025/ds722/resources).

## Problem 1: Automatic Differentiation with PyTorch

In this exercise, you'll explore how PyTorch builds computational graphs and computes gradients automatically using `autograd`.

### Objectives

- Define scalar and vector functions using PyTorch tensors.
- Use `.backward()` to compute gradients.
- Visualize the computational graph and understand gradient propagation.

### Step 1: Import PyTorch

In [ ]:
import torch

### Step 2: Scalar Function

Below we illustrate how to use Pytorch autograd to compute $f(2)$, and $f^{\prime}(2)$ of $f(x)=x^{2} + 3x + 2$.

In [ ]:
# Create a tensor with requires_grad=True to track computation
x = torch.tensor(2.0, requires_grad=True)

# Define the function
f = x**2 + 3*x + 2

# Compute the gradient
f.backward()

# Print the gradient df/dx
print("x =", x.item())
print("f(x) =", f.item())
print("df/dx =", x.grad.item())

### Step 3: Computational Graph

Each operation in $f(x)$ on the tensor $x=2$ creates a node in the computational graph:

- $c = 3\cdot x$ (Multiplication)
- $b = x\cdot x$ (Power)
- $a = b + c$ (Add)
- $f = a + 2$ (Add)

Using the `grad_fn` and `next_function` attributes, we can see the inner workings of the Pytorch data structures responsible for storing and calculating the partial derivatives to calculate the gradients.

It is an efficient way to compute $\nabla f(x)$ by first computing

1. $df/da$
1. $da/db$ and $da/dc$
1. $db/dx$ and $dc/dx$

then $\nabla f(x) = (df/da)(da/db)(db/dx) + (df/da)(da/dc)(dc/dx)$.


Execute the following cell to observe this behavior.

In [ ]:
# AddBackward object for adding the two quantities f = a + 2
print("f.grad_fn:", f.grad_fn)
# AddBackward object for adding the two quantities a = b + c and None for the scalar 2
print("f.grad_fn.next_functions:", f.grad_fn.next_functions)
# PowBackward object for b = x**2 and MulBackward for c=3*x
print("f.grad_fn.next_functions:", f.grad_fn.next_functions[0][0].next_functions)
# AccumulateGrad object is a special node to tell the program  to store the calculated gradient of x when backward is called
# Leaf node corresponding to the tensor x
print("f.grad_fn.next_functions:", f.grad_fn.next_functions[0][0].next_functions[0][0].next_functions)
# AccumulateGrad object is a placeholder for x and None for scalar 3
print("f.grad_fn.next_functions:", f.grad_fn.next_functions[0][0].next_functions[1][0].next_functions)

### Step 4: Vector Function

Create similar code as in Step 2 for the vector function $f(x,y)=x^{2}y + \sin{y}$.

In [ ]:
#TODO

### Step 5: Computational Graph

Create similar code as in Step 3 for the vector function $f(x,y)=x^{2}y + \sin{y}$.

Is it becoming clearer how Pytorch computes the gradients of functions and how autograd works?

In [69]:
#TODO

## Problem 2: Numerical Integration

In this problem we investigate two methods for numerical integration:

- the trapezoid rule
- Simpson's rule


## Trapezoid Rule

The Trapezoid Rule approximates the area under the curve using trapezoids:

$$
\int_a^b f(x)\,dx \approx \frac{h}{2} \left[ f(x_0) + 2 \sum_{i=1}^{n-1} f(x_i) + f(x_n) \right]
$$

where:
- $ h = \frac{b - a}{n} $
- $ x_i = a + i h $ for $ i = 0, 1, \dots, n $

## Midpoint Rule

The Midpoint Rule approximates the definite integral by evaluating the function at the midpoint of each subinterval:

$$
\int_a^b f(x)\,dx \approx h \sum_{i=0}^{n-1} f\left(x_i + \frac{h}{2}\right)
$$

where:
- $ h = \frac{b - a}{n} $ is the width of each subinterval
- $ x_i = a + i h $ is the start of the $i$-th subinterval

Your job is to

1. Implement these rules as Python functions. 
1. Use your Python functions to approximate the integrals of
    - $f_{1}(x) = \sin{x}$ on $[0, \pi]$
    - $f_{2}(x) = x^{5}$ on $[0, 1]$
1. Compute the error between your approximations and the true value of the integral as you double the number of points in your approximation. Plot the errors on a loglog plot. What does the slope of the line tell you about the error?

In [ ]:
#TODO
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# Trapezoid Rule Implementation
# -------------------------------
def trapezoid_rule(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    return h * (0.5 * f(x[0]) + np.sum(f(x[1:-1])) + 0.5 * f(x[-1]))

# -------------------------------
# Simpson's Rule Implementation
# -------------------------------
def simpsons_rule(f, a, b, n):
    if n % 2 != 0:
        raise ValueError("Simpson's rule requires an even number of intervals.")
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = f(x)
    return (h / 3) * (fx[0] + fx[-1] + 2 * np.sum(fx[2:n:2]) + 4 * np.sum(fx[1:n:2]))

# -------------------------------
# Functions to Integrate
# -------------------------------
def f1(x):
    return np.sin(x)

def f2(x):
    return x**5

# Exact integrals
exact_f1 = 2.0          
exact_f2 = 1.0 / 6      

# -------------------------------
# Error Analysis
# -------------------------------
ns = [4, 8, 16, 32, 64, 128]
errors_f1_trap = []
errors_f1_simp = []
errors_f2_trap = []
errors_f2_simp = []

for n in ns:
    # f1: sin(x) on [0, pi]
    trap_f1 = trapezoid_rule(f1, 0, np.pi, n)
    simp_f1 = simpsons_rule(f1, 0, np.pi, n)
    errors_f1_trap.append(abs(trap_f1 - exact_f1))
    errors_f1_simp.append(abs(simp_f1 - exact_f1))
    
    # f2: x^5 on [0, 1]
    trap_f2 = trapezoid_rule(f2, 0, 1, n)
    simp_f2 = simpsons_rule(f2, 0, 1, n)
    errors_f2_trap.append(abs(trap_f2 - exact_f2))
    errors_f2_simp.append(abs(simp_f2 - exact_f2))

# -------------------------------
# Plotting Error Comparison
# -------------------------------
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.loglog(ns, errors_f1_trap, 's-', label='Trapezoid Rule')
plt.loglog(ns, errors_f1_simp, 'o-', label="Simpson's Rule")
plt.title(r'Error for $f_1(x) = \sin(x)$')
plt.xlabel('Number of Subintervals (n)')
plt.ylabel('Absolute Error')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.loglog(ns, errors_f2_trap, 's-', label='Trapezoid Rule')
plt.loglog(ns, errors_f2_simp, 'o-', label="Simpson's Rule")
plt.title(r'Error for $f_2(x) = x^5$')
plt.xlabel('Number of Subintervals (n)')
plt.ylabel('Absolute Error')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

3. The slope of the line is the order of convergence. For every doubling of points, the error goes down by a factor of 2 in the trapezoid rule. For Simpson's rule, when you double the number of points, the error decreases by a factor of 4.